<a href="https://colab.research.google.com/github/josegabrielguerrero26/CSE450-machinelearning/blob/main/project2/starter_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ** INTRODUCTION**

1. Data Dictionary: https://byui-cse.github.io/cse450-course/module-03/housing-dictionary.txt
2. Problem Intro: https://byui-cse.github.io/cse450-course/module-03/intro.html
3. Dataset https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv

-You’re working as a data analyst for Reddic Housing LLC, a company that specializes in providing house price estimates to development firms.

Recently, a rival company (Grenic Housing Inc) announced that they were launching a set of initiatives to use Artificial Intelligence and Machine Learning to bring their housing estimates into the twenty-first century.

After a heated shareholders meeting, the board of directors has tasked the CEO with making sure Reddic Housing LLC isn’t left behind.

# Questions

Questions
You're about to go into a strategy meeting with the CEO, Vice President of Human Resources, and Vice President of Finance. They want to make sure you have the data required to answer the questions they're most interested in.

Be prepared to answer the following questions:

- **PROBLEM TYPE**

DEVON, THE CEO SAYS:
I just sat through four hours of machine learning training with the board of directors this past week, so I'm curious to get your take on this.

Looking at the data and our business model, what kind of machine learning problem do you think we're looking at here?

- **MODEL CONFIDENCE**

CECIL, THE VP OF CUSTOMER RELATIONS ASKS:
My biggest concern right now is making sure that whatever method we come up with to predict housing prices, we can also attach some kind of empirical confidence metric.

Based on your initial analysis of the data, your team feels you can best show confidence in your model by using:

The sum of squares error (SSE).
The mean squared error (MSE).
The root mean squared error (RMSE).
The 2R value.

- **INSURANCE QUESTION**

WILLIAM, THE VP OF FINANCE ASKS:
Our insurance customers are particularly interested in making sure that homes in… unsavory neighborhoods, aren't estimated high.

Is there a way we can easily identify properties in low income areas and have the model lower those estimates to protect our insurance customers' interests?

Based on your initial analysis of the data, your team feels:

We can lower the predicted price for specific neighborhoods before training the model.
We can add average income or other demographic information for the area as features.
For specific zip codes we could add a step to our pipeline that reduces the predicted price by a specific percentage prior to outputting the final price result to ensure the properties aren’t being overvalued.
Taking this kind of action would be a violation of federal laws and/or ethics.

- **DATA ANALYSIS**

JOHNNY, THE DATA SCIENCE INTERN ASKS:
The head of data science says we should use gradient boosted trees for this analysis.

I've noticed that a lot of the features use pretty different ranges.

For example, how should we handle square footage?

# Libraries- Dataset

In [1]:
#!pip install ydata_profiling # data Table Analysis
#!pip install typing_extensions # data Table Analysis
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt # for graphs
from sklearn.model_selection import train_test_split #Model Split to train
from sklearn.metrics import accuracy_score           #Accuracy model


housing = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')

# Table Description

# Data Wrangling - Missing Values, transformation

# Machine Learning Model

## Split Date

## Imbalance

## Training Model

## Resample

## Prediction

## Evaluation

# Applying Model